In [ ]:
"""
Dynamic Hospital Chatbot (console) with Unknown Department Option
"""

import requests
import sys
import time

BASE_URL = "https://hospital-analytics-db.onrender.com"  # change if needed
HOSPITAL_CONTACT = "+91 98765 43210"


# ---------- Helpers ----------
def safe_get(record, *keys, default=None):
    for k in keys:
        if isinstance(record, dict) and k in record:
            return record[k]
    return default


def fetch_json(path):
    try:
        r = requests.get(f"{BASE_URL}{path}", timeout=REQUEST_TIMEOUT)
        r.raise_for_status()
        return r.json()
    except requests.RequestException as e:
        print(f"\n❌ Network/API error when calling {path}: {e}")
        return None


def load_departments():
    data = fetch_json("/departments")
    if data is None:
        return None
    depts = []
    for rec in data:
        dept_id = safe_get(rec, "department_id", "id", "dept_id")
        name = safe_get(rec, "department_name", "name")
        doctors_field = safe_get(rec, "doctors")
        head_doctor = safe_get(rec, "head_doctor")
        depts.append({
            "department_id": int(dept_id) if dept_id is not None else None,
            "department_name": name,
            "doctors_raw": doctors_field,
            "head_doctor": head_doctor,
            "raw": rec
        })
    return depts


def load_disease_symptoms():
    data = fetch_json("/disease_symptoms")
    if data is None:
        return None
    records = []
    for rec in data:
        disease = safe_get(rec, "disease", "disease_name", "name")
        symptoms_raw = safe_get(rec, "symptoms", "symptom_list", "") or ""
        dept_id = safe_get(rec, "department_id", "dept_id")
        dept_name = safe_get(rec, "department_name", "dept_name")
        symptoms = [s.strip().lower() for s in symptoms_raw.split(",") if s.strip()]
        records.append({
            "disease": disease,
            "symptoms": symptoms,
            "department_id": int(dept_id) if dept_id is not None else None,
            "department_name": dept_name
        })
    return records


def build_dept_to_diseases(disease_records):
    mapping = {}
    for rec in disease_records:
        dept = rec["department_id"]
        if dept not in mapping:
            mapping[dept] = []
        mapping[dept].append(rec)
    return mapping


def build_unique_symptoms_for_dept(disease_list):
    seen = []
    for d in disease_list:
        for s in d["symptoms"]:
            if s not in seen:
                seen.append(s)
    return seen


def get_department_by_id(departments, dept_id):
    return next((d for d in departments if d["department_id"] == dept_id), None)


def get_doctors_from_department_record(dept_record):
    if not dept_record:
        return []
    raw = dept_record.get("doctors_raw")
    if raw:
        if isinstance(raw, list):
            return [str(x).strip() for x in raw if str(x).strip()]
        return [x.strip() for x in str(raw).split(",") if x.strip()]
    head = dept_record.get("head_doctor")
    if head:
        return [head.strip()]
    rawdict = dept_record.get("raw", {})
    possible = safe_get(rawdict, "doctor", "head", "lead_doctor")
    if possible:
        return [x.strip() for x in str(possible).split(",") if x.strip()]
    return []


def pretty_pct(x):
    return f"{x:.1f}%"


# ---------- Probability Computation ----------
def compute_probabilities_for_dept(selected_symptoms, disease_list):
    results = []
    sel_set = set(s.strip().lower() for s in selected_symptoms)
    for d in disease_list:
        disease_symptoms = [s.strip().lower() for s in d.get("symptoms", [])]
        if not disease_symptoms:
            continue
        overlap = len(sel_set.intersection(disease_symptoms))
        if overlap > 0:
            prob = (overlap / len(disease_symptoms)) * 100
            results.append({
                "disease": d.get("disease") or d.get("disease_name"),
                "probability": round(prob, 2),
                "matched": overlap,
                "total_symptoms": len(disease_symptoms),
                "department_id": d.get("department_id")
            })
    results.sort(key=lambda x: x["probability"], reverse=True)
    return results


# ---------- Utility: Matrix Printer ----------
def display_symptoms_matrix(symptom_list, cols=4):
    """Print symptoms in table format"""
    for i, s in enumerate(symptom_list, 1):
        print(f"{i:2}. {s:<20}", end="  ")
        if i % cols == 0:
            print()
    print()


# ---------- New: Global Symptom Matrix ----------
def build_global_symptom_matrix(diseases):
    matrix = {}
    for d in diseases:
        dept = d.get("department_name") or "Unknown Department"
        for s in d.get("symptoms", []):
            matrix.setdefault(dept, set()).add(s)
    return matrix


def global_symptom_flow(name, age, diseases, departments, dept_map):
    matrix = build_global_symptom_matrix(diseases)

    # Flatten into numbered options
    all_symptoms = []
    print("\n🌐 Global Symptom Matrix (all departments):")
    idx = 1
    for dept, syms in matrix.items():
        print(f"\n{dept}:")
        syms = sorted(syms)
        for i, s in enumerate(syms, 1):
            print(f"{idx:2}. {s:<20}", end="  ")
            if i % 4 == 0:
                print()
            all_symptoms.append((idx, s, dept))
            idx += 1
        print()

    raw = input("\nEnter symptom numbers (comma-separated): ").strip()
    try:
        nums = [int(x.strip()) for x in raw.split(",") if x.strip()]
    except ValueError:
        print("❌ Invalid input.")
        return

    selected_symptoms = [s for (i, s, d) in all_symptoms if i in nums]
    if not selected_symptoms:
        print("⚠️ No valid symptoms chosen.")
        return

    results = compute_probabilities_for_dept(selected_symptoms, diseases)
    if not results:
        print("\n❌ No matching disease found.")
        return

    print("\n🧾 Possible diagnoses (ranked):")
    for r in results:
        print(f"- {r['disease']}: {pretty_pct(r['probability'])} (matched {r['matched']}/{r['total_symptoms']})")

    top = results[0]
    dept = dept_map.get(top["department_id"])
    dept_name = dept["department_name"] if dept else "Unknown Department"
    print(f"\n✅ Most likely: {top['disease']} ({pretty_pct(top['probability'])})")
    print(f"➡ Department: {dept_name}")

    doctors = get_doctors_from_department_record(dept)
    if doctors:
        print("\n👩‍⚕️ Doctors:")
        for doc in doctors:
            print(f"- {doc}")

    print(f"\n📞 Hospital contact: {HOSPITAL_CONTACT}")
    book = input("\nDo you want to book an appointment? (yes/no): ").strip().lower()
    if book == "yes":
        register_patient_interactive(name, age, top["disease"], top["department_id"])
    print("\n✅ Have a healthy and happy day. Thank you!")


# ---------- Conversation ----------
def run_chatbot():
    print("\n🏥 Hospital Assistant (console)\n")
    name = input("Enter your full name: ").strip()
    age = input("Enter your age: ").strip()

    departments = load_departments()
    diseases = load_disease_symptoms()
    if departments is None or diseases is None:
        print("\n❌ Unable to load data.")
        return

    dept_to_diseases = build_dept_to_diseases(diseases)
    dept_map = {d["department_id"]: d for d in departments}

    while True:
        print("\n📋 Departments:")
        for d in departments:
            print(f"{d['department_id']}. {d['department_name']}")
        print("999.Confused or unsure about your symptoms related department enter 999 (symptoms from all departments):")
        print("0. Exit")

        try:
            dep_choice = int(input("\nSelect department number: ").strip())
        except ValueError:
            print("❌ Invalid input.")
            continue

        if dep_choice == 0:
            print("\n✅ Have a healthy and happy day. Thank you!")
            return
        if dep_choice == 999:
            global_symptom_flow(name, age, diseases, departments, dept_map)
            return
        if dep_choice not in dept_map:
            print("❌ Department not found.")
            continue

        disease_list = dept_to_diseases.get(dep_choice, [])
        if not disease_list:
            print("⚠️ No data for this department.")
            continue

        symptom_list = build_unique_symptoms_for_dept(disease_list)
        print(f"\nYou selected: {dept_map[dep_choice]['department_name']}")
        display_symptoms_matrix(symptom_list)

        raw = input("\nEnter symptoms (numbers comma-separated): ").strip()
        try:
            nums = [int(x.strip()) for x in raw.split(",") if x.strip()]
        except ValueError:
            print("❌ Invalid input.")
            continue

        selected_symptoms = [symptom_list[n-1].lower() for n in nums if 1 <= n <= len(symptom_list)]
        if not selected_symptoms:
            print("⚠️ No valid symptoms chosen.")
            continue

        results = compute_probabilities_for_dept(selected_symptoms, disease_list)
        if not results:
            print("\n❌ No matching disease found.")
            continue

        print("\n🧾 Possible diagnoses (ranked):")
        for r in results:
            print(f"- {r['disease']}: {pretty_pct(r['probability'])} (matched {r['matched']}/{r['total_symptoms']})")

        top = results[0]
        dept_name = dept_map[dep_choice]["department_name"]
        print(f"\n✅ Most likely: {top['disease']} ({pretty_pct(top['probability'])})")
        print(f"➡ Department: {dept_name}")

        doctors = get_doctors_from_department_record(dept_map[dep_choice])
        if doctors:
            print("\n👩‍⚕️ Doctors:")
            for doc in doctors:
                print(f"- {doc}")

        print(f"\n📞 Hospital contact: {HOSPITAL_CONTACT}")
        book = input("\nDo you want to book an appointment? (yes/no): ").strip().lower()
        if book == "yes":
          print("\n📞 Please call hospital reception at: +91-9876543210")
          print(f"NOTE: Respected {name} kindly tell the name of {doc} to the receptionist to book your appointment.")
          print("\n✅ Thank you for choosing us. Have a healthy and happy day. Thank you!")
          break
        else:
          print("\n✅ Thank you for contacting us. Have a healthy and happy day. Thank you!")
          break
if __name__ == "__main__":
    run_chatbot()



🏥 Hospital Assistant (console)

Enter your full name: sai
Enter your age: 20

📋 Departments:
1. General Medicine
2. Cardiology
3. Orthopedics
4. Neurology
5. Pediatrics
6. Gynecology
7. Emergency
8. ICU
9. Oncology
10. Dermatology
999.Confused or unsure about your symptoms related department enter 999 (symptoms from all departments):
0. Exit

Select department number: 4

You selected: Neurology
 1. sudden weakness        2. speech difficulty      3. vision problems        4. dizziness             
 5. severe headache        6. nausea                 7. sensitivity to light   8. seizures              
 9. loss of consciousness  10. confusion             11. twitching             12. tremors               
13. slow movement         14. stiff muscles         15. balance problems      16. memory loss           
17. difficulty speaking   18. mood swings           19. numbness              20. fatigue               
21. muscle weakness       

Enter symptoms (numbers comma-separated): 3,4,1